# Reward Model testing pipeline

In [ ]:
import logging
import os
import huggingface_hub

import chai_guanaco as chai

from chaiverse.dev.dataset import DatasetLoader, RewardDatasetBuilder
from chaiverse.dev.tokenizer import GPT2Tokenizer
from chaiverse.dev.model.reward_model import RewardClassificationTrainer, RewardRegressionTrainer

## login setup

In [ ]:
chai.developer_login()

In [ ]:
huggingface_hub.login()

In [ ]:
os.environ["WANDB_DISABLED"] = "true"
logging.basicConfig(level=logging.INFO)

## Load and process a small dataset

In [ ]:
# load data
data_path = 'ChaiML/20231012_chai_prize_reward_model_data'
data_loader = DatasetLoader(
        hf_path=data_path,
        data_samples=100,
        validation_split_size=0.1,
        shuffle=True,
        )
df = data_loader.load()
print(df)

In [ ]:
# process data
tokenizer_loader = GPT2Tokenizer(
        padding_side='right',
        truncation_side='left',
        )
data_builder = RewardDatasetBuilder(
        tokenizer_loader=tokenizer_loader,
        block_size=512,
        )
data = data_builder.generate(df)
print(data)

## model setup and fitting

In [ ]:
# train model
model = RewardClassificationTrainer(
        model_name='gpt2',
        tokenizer_loader=tokenizer_loader,
        output_dir='test_reward_model',
        # num_labels=2,
        learning_rate=1e-5,
        num_train_epochs=1,
        bf16=False,
        logging_strategy='steps',
        logging_steps=2,
        eval_strategy='steps',
        eval_steps=2
        )

In [ ]:
model.fit(data)

## upload model

In [ ]:
#your hugging-face model path
model_path = ''
model.push_to_hub(model_path)